In [1]:
import pandas as pd
from io import BytesIO
import requests
import random

def fetch_data(url, skiprows=0):
    r = requests.get(url)
    data = r.content

    return pd.read_csv(BytesIO(data), index_col=0, skiprows=skiprows)

catalog = fetch_data("https://docs.google.com/spreadsheets/d/1JyGlqmLg9k7UubOw-V_CC8McRZxn2PtknvsrMIxvLGk/export?gid=0&format=csv", 2)
data_interp_catalog = fetch_data("https://docs.google.com/spreadsheets/d/1JyGlqmLg9k7UubOw-V_CC8McRZxn2PtknvsrMIxvLGk/export?gid=1340046923&format=csv", 0)

completed_sets = fetch_data("https://docs.google.com/spreadsheets/d/1JyGlqmLg9k7UubOw-V_CC8McRZxn2PtknvsrMIxvLGk/export?gid=1791033131&format=csv")
completed_suppl = fetch_data("https://docs.google.com/spreadsheets/d/1JyGlqmLg9k7UubOw-V_CC8McRZxn2PtknvsrMIxvLGk/export?gid=399306386&format=csv")

In [6]:
categories = catalog.keys()[7:]
repl_nan = { i : False for i in catalog.keys()[3:]}
repl_checked = { i : 'x' for i in catalog.keys()[3:]}

scrubbed_cat = catalog.fillna(value=repl_nan).replace(repl_checked, True)
scrubbed_cat.head()

,Local Id,Source,Type,AR,ALG,DA,GEO,ARG,FAD,PCT,...,PROB,PRS,CC,TRI,CG,MG,WP,2WP,VC,AQ
Global Id,,,,,,,,,,,,,,,,,,,,,
50,7.01,M,QC,True,False,False,False,True,False,False,...,False,False,False,False,False,False,False,False,False,False
51,7.02,M,QC,True,False,False,False,True,False,False,...,False,False,False,False,False,False,False,False,False,False
52,7.03,M,QC,True,False,False,False,True,False,False,...,False,False,False,False,False,False,False,False,False,False
53,7.04,M,NE,True,False,False,False,True,False,False,...,False,False,False,False,False,False,False,False,False,False
54,7.05,M,QC,True,False,False,False,True,False,False,...,False,False,False,False,False,False,False,False,False,False


In [7]:
#scrubbed_cat.set_index('Global Id', inplace=True)
#completed_sets.set_index('Global Id', inplace=True)
#completed_suppl.set_index('Global Id', inplace=True)
#scrubbed_cat

In [8]:
categorical_cols = scrubbed_cat.columns[3:7]
ctotals = { col:scrubbed_cat[col].where(scrubbed_cat[col]==True).count() for col in categorical_cols }
ctotals

{'AR': 233, 'ALG': 169, 'DA': 115, 'GEO': 132}

In [9]:
skillset_cols = scrubbed_cat.columns[7:-4]
    
stotals = { col:scrubbed_cat[col].where(scrubbed_cat[col]==True).count() for col in skillset_cols }
stotals

{'ARG': 29,
 'FAD': 40,
 'PCT': 54,
 'NP': 44,
 'DAP': 34,
 'EAR': 38,
 'EAF': 35,
 'IAV': 44,
 'FFS': 58,
 'ROW': 30,
 'RAT': 41,
 'CT': 37,
 'DST': 31,
 'PROB': 47,
 'PRS': 22,
 'CC': 26,
 'TRI': 43,
 'CG': 24,
 'MG': 18}

In [10]:
misc_cols = scrubbed_cat.columns[-4:]
mtotals = { col:scrubbed_cat[col].where(scrubbed_cat[col]==True).count() for col in misc_cols }
mtotals

{'WP': 36, '2WP': 25, 'VC': 20, 'AQ': 42}

In [11]:
scrubbed_cat.where(scrubbed_cat["ARG"]==True).count()

Local Id    29
Source      29
Type        29
AR          29
ALG         29
DA          29
GEO         29
ARG         29
FAD         29
PCT         29
NP          29
DAP         29
EAR         29
EAF         29
IAV         29
FFS         29
ROW         29
RAT         29
CT          29
DST         29
PROB        29
PRS         29
CC          29
TRI         29
CG          29
MG          29
WP          29
2WP         29
VC          29
AQ          29
dtype: int64

In [310]:
scrubbed_cat.where(scrubbed_cat['Type'] == 'QC').dropna().sample(7)
scrubbed_cat.where(scrubbed_cat['Type'] == 'NE').dropna().sample(2)
scrubbed_cat.where(scrubbed_cat['GEO'] == True).where(scrubbed_cat['Type'] != 'QC').dropna().sample(2)

,Local Id,Source,Type,AR,ALG,DA,GEO,ARG,FAD,PCT,...,PROB,PRS,CC,TRI,CG,MG,WP,2WP,VC,AQ
Global Id,,,,,,,,,,,,,,,,,,,,,
738,27.35,M,MC,0.0,False,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
781,29.12,M,NE,0.0,False,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


In [311]:
completed_full = pd.concat([completed_sets, completed_suppl], ignore_index=False, sort=False).drop_duplicates().set_index('Global Id')
j = scrubbed_cat.join(completed_full, how='left', lsuffix='_left', rsuffix='_right', sort=False)
remaining_qs = j[j['Local Id_right'].isna()]
remaining_qs

,Local Id_left,Source,Type,AR,ALG,DA,GEO,ARG,FAD,PCT,...,TRI,CG,MG,WP,2WP,VC,AQ,Local Id_right,Completed,Correct
Global Id,,,,,,,,,,,,,,,,,,,,,
50,7.01,M,QC,True,False,False,False,True,False,False,...,False,False,False,False,False,False,False,NaN,NaN,NaN
51,7.02,M,QC,True,False,False,False,True,False,False,...,False,False,False,False,False,False,False,NaN,NaN,NaN
52,7.03,M,QC,True,False,False,False,True,False,False,...,False,False,False,False,False,False,False,NaN,NaN,NaN
53,7.04,M,NE,True,False,False,False,True,False,False,...,False,False,False,False,False,False,False,NaN,NaN,NaN
54,7.05,M,QC,True,False,False,False,True,False,False,...,False,False,False,False,False,False,False,NaN,NaN,NaN
55,7.06,M,QC,True,False,False,False,True,False,False,...,False,False,False,False,False,False,False,NaN,NaN,NaN
56,7.07,M,QC,True,False,False,False,True,False,False,...,False,False,False,False,False,False,False,NaN,NaN,NaN
57,7.08,M,QC,True,False,False,False,True,False,False,...,False,False,False,False,False,False,False,NaN,NaN,NaN
60,7.11,M,QC,True,False,False,False,True,False,False,...,False,False,False,False,False,False,False,NaN,NaN,NaN


In [312]:
completed_full[completed_full.Correct == True].Correct.count() / completed_full.count()

Local Id     0.893939
Completed    0.983333
Correct      0.893939
dtype: float64

The following will calculate how many questions of each type/format (quantitative comparison, multiple choice, etc.) exists in the remaining question catalog. Our goal is to randomly sample across these types such that our generated tests closely mimmick the general structure of a real test. It will be fairly hard to get this exact, mostly because our practice test sample is small (8, 20 question sections at the time of writing) and we are also generating a fairly small sample of generated section (20ish). This is not really much of a concern though as the test is fairly formulaic in design. Each section always begins with 8 or 7 quantitative comparison questions and never has more than 1 or 2 numerical entry and multi-answer questions. Further, there is always 3 data interpretation questions which are always multiple choice or multiple answer.

Note: the final result is normalized to be a percent of the whole catalog.

In [313]:
x = remaining_qs.groupby('Type').sum()[categories].sum(axis=1)
x / x.sum()

Type
MA    0.060606
MC    0.445323
NE    0.158103
QC    0.335968
dtype: float64

This breakdown based type differs a little to [the one we discerned from the practice sets](http://localhost:8888/notebooks/Documents/git/gre_analysis/Catalog.ipynb#) (I've hardcoded the results in the dictionary below). We will "rebalance" the proportions to match those of the practice tests by randomly discarding a few questions from "over-allocated" types.

In [314]:
test_weights = {
    'MA': 0.060606,
    'MC': 0.445323,
    'NE': 0.158103,
    'QC': 0.335968
}

#error = {}
#for key in remaining_weights.to_dict(): 
#    error[key] = remaining_weights[key] - test_weights[key] 
#lo_to_hi = sorted(error.items(), key=lambda kv: kv[1])
   

In [403]:
def char_range(c1, c2):
    """Generates the characters from `c1` to `c2`, inclusive."""
    for c in xrange(ord(c1), ord(c2)+1):
        yield chr(c)
        
def generate_test_samples(num_tests, catalog, data_interp_catalog, type_target_weights):
    rem_qs_by_type = remaining_qs.groupby('Type').sum()[categories].sum(axis=1)
    
    target_qs_quant = {}
    for typ, weight in type_target_weights.items():
        target_size_for_type = weight * 20
        target_qs_quant[typ] = round(target_size_for_type) # TBD could result in off-by-one on different dataset
        assert(target_size_for_type < rem_qs_by_type[typ])
        
    assert(sum(target_qs_quant.values()))   
    
    tests = []
    for i in range(num_tests):
        test_qs = {
            'MA': {},
            'MC': {},
            'NE': {},
            'QC': {},
            'DI': {}
        }
        
        interp_qs = data_interp_catalog[data_interp_catalog['Set Id'] == chr(ord('A') + i)]
        if (interp_qs.count()[1] >= 3):
            interp_qs = interp_qs.sample(n=3)
        
        adjusted_quant = target_qs_quant.copy()
        for index, row in interp_qs.iterrows():
            typ = row['Type']
            
            test_qs['DI'].update({index: row['Local Id']})
            
            adjusted_quant[typ] -= 1
            if (adjusted_quant[typ] < 0):
                adjusted_quant['NE'] -= abs(adjusted_quant[typ])
                adjusted_quant[typ] = 0    
                if (adjusted_quant['NE'] < 0):
                    adjusted_quant['MC'] -= abs(adjusted_quant['NE'])
                    adjusted_quant['NE'] = 0
        
        
        for typ, quant in adjusted_quant.items():
            test_qs[typ].update(remaining_qs[remaining_qs['Type'] == typ].sample(n=quant)['Local Id_left'].to_dict())
        
        tests.append(test_qs)
        
    return tests

def flatten_and_shuffle(test_samples):
    rows = []
    for i, qs_by_type in enumerate(test_samples):
        qcs = list(qs_by_type['QC'].items())
        random.shuffle(qcs)
        
        dis = list(qs_by_type['DI'].items())
        random.shuffle(dis)
        
        others = list(qs_by_type['MC'].items()) + list(qs_by_type['MA'].items()) + list(qs_by_type['NE'].items())
        random.shuffle(others)
        
        test_set = qcs + others[:-3] + dis + others[-3:]
        
        rows += [(i, q[0], q[1]) for q in test_set]
    return pd.DataFrame(rows, columns =['Set Id', 'Global Id', 'Local Id']) 

samples = generate_test_samples(20, remaining_qs, data_interp_catalog, test_weights)
tests = flatten_and_shuffle(samples)
tests

,Set Id,Global Id,Local Id
0,0,123,9.11
1,0,218,11.08
2,0,474,18.17
3,0,140,9.28
4,0,829,30.42
5,0,790,30.03
6,0,818,30.31
7,0,203,10.50
8,0,439,17.07
9,0,142,9.30


In [404]:
tests.to_csv(r'C:\Users\bille\Documents\git\gre_analysis\test_gen.csv', index=False)